In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import time
import os
import pickle

from plotly.subplots import make_subplots
from my_lib.utility import *
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

## Actividad 3: Algoritmo genético para resolver el problema del viajero frecuente

El problema es de tipo TSP con 18 ciudades del conteninte Americano. Por lo tanto existen $18!\$ soluciones posibles ($6402373705728000$), dada el basto espacio de busqueda la población de individuos se debe limitar a minimo 100 y máximo 500.

> Criterios para la evaluación
> - Implementar 2 métodos de cruza, PMX y algún otro método como OX o CX
> - Implementar 2 métodos de mutación, Scramble y Heuristic
> - Implementar un método de selección, cualquiera de los ya vistos
> - Implementar un método hibrido de paro
> - Implementar un méotodo elitista
> - Realizar las comparaciones y la discusión de los resultados

In [2]:
max_gen_ = 40

# Codificación del dominio

In [3]:
ciudades18 = {
    'Ciudad': ['Mexico City', 'Merida', 'Buenos Aires', 'Quito', 'Washington D.C.', 
               'New York', 'Miami', 'Monterrey', 'Panama City', 'San Salvador',
               'Managua', 'Brazilia', 'Mendoza', 'Caracas', 'Montevideo', 
               'Guadalajara', 'Boston', 'Bogotá'],
    'Latitud': [19.4326, 20.9674, -34.6037, -0.1807, 38.9072, 
                40.7128, 25.7617, 25.6866, 8.9824, 13.6929, 
                12.1140, -15.8267, -32.8895, 10.4806, -34.9011, 
                20.6597, 42.3601, 4.7110],
    'Longitud': [-99.1332, -89.5926, -58.3816, -78.4678, -77.0369, 
                 -74.0060, -80.1918, -100.3161, -79.5199, -89.2182, 
                 -86.2362, -47.9218, -68.8458, -66.9036, -56.1645, 
                 -103.3496, -71.0589, -74.0721]
} 
nombre_ciudades = np.array(ciudades18['Ciudad'])

In [4]:
id_ciudades = np.arange(nombre_ciudades.shape[0])
bin_ciudades = binarizar(datos=id_ciudades,nbites=3)

ciudades = np.zeros((nombre_ciudades.shape[0],4),dtype="object")
ciudades[:,0] = nombre_ciudades
ciudades[:,1] = id_ciudades
ciudades[:,2] = bin_ciudades
ciudades[:,3] = np.zeros((nombre_ciudades.shape[0]))
print(ciudades)

[['Mexico City' 0 '000' 0.0]
 ['Merida' 1 '001' 0.0]
 ['Buenos Aires' 2 '010' 0.0]
 ['Quito' 3 '011' 0.0]
 ['Washington D.C.' 4 '100' 0.0]
 ['New York' 5 '101' 0.0]
 ['Miami' 6 '110' 0.0]
 ['Monterrey' 7 '111' 0.0]
 ['Panama City' 8 '1000' 0.0]
 ['San Salvador' 9 '1001' 0.0]
 ['Managua' 10 '1010' 0.0]
 ['Brazilia' 11 '1011' 0.0]
 ['Mendoza' 12 '1100' 0.0]
 ['Caracas' 13 '1101' 0.0]
 ['Montevideo' 14 '1110' 0.0]
 ['Guadalajara' 15 '1111' 0.0]
 ['Boston' 16 '10000' 0.0]
 ['Bogotá' 17 '10001' 0.0]]


## Matriz de distancias

#### Fórmula de Haversine 

$d = 2r \cdot arsin(\sqrt{sin^2(\frac{\Delta\phi}{2}) + cos(\phi_1) \cdot cos(\phi_2) \cdot sin^2(\frac{\Delta\lambda}{2})})$

* d es la distancia entre los dos puntos 
* r = radio de la tierra (6371)
* $\phi_1, \phi_2$ = latitudes de los puntos en rad
* $\Delta\phi$ = diferencia de las latitudes
* $\Delta\lambda$ = diferencia de las longitudes


In [5]:
# Latitud y longitud de cada ciudad

n_ciudades = ciudades.shape[0]
distancias = np.zeros((n_ciudades, n_ciudades))

for i in range(n_ciudades):
    c1_latitud = ciudades18["Latitud"][i]
    c1_longitud = ciudades18["Longitud"][i]
    for j in range(n_ciudades):
        if i != j:
            c2_latitud = ciudades18["Latitud"][j]
            c2_longitud = ciudades18["Longitud"][j]
            distancias[i][j] = get_distancia(
                lat1=c1_latitud, long1=c1_longitud, lat2=c2_latitud, long2=c2_longitud
            )

distancias_df = pd.DataFrame(distancias,columns=ciudades18["Ciudad"],index=ciudades18["Ciudad"])
display(distancias_df)

,Mexico City,Merida,Buenos Aires,Quito,Washington D.C.,New York,Miami,Monterrey,Panama City,San Salvador,Managua,Brazilia,Mendoza,Caracas,Montevideo,Guadalajara,Boston,Bogotá
Mexico City,0.000000,1009.959843,7392.178684,3135.709015,3031.811604,3359.383041,2065.272960,705.925938,2408.857647,1233.956557,1600.957943,6835.065741,6648.609671,3595.449156,7554.599238,461.067374,3665.482788,3170.731318
Merida,1009.959843,0.000000,7004.211339,2644.150812,2327.565260,2643.050506,1097.212425,1213.520720,1715.030837,809.861482,1047.292813,6116.463263,6381.173335,2689.070020,7144.536815,1429.853511,2941.198434,2464.943628
Buenos Aires,7392.178684,7004.211339,0.000000,4360.060643,8396.527908,8526.194114,7094.952486,8028.590573,5333.037977,6280.332978,5968.507501,2334.403612,985.614665,5093.044534,205.232076,7766.608993,8654.780454,4670.520317
Quito,3135.709015,2644.150812,4360.060643,0.000000,4348.827501,4569.460001,2890.594533,3711.248568,1025.530802,1944.679440,1613.735904,3776.161246,3774.047997,1743.741681,4501.636863,3562.151769,4788.167318,730.923189
Washington D.C.,3031.811604,2327.565260,8396.527908,4348.827501,0.000000,327.582714,1491.176569,2623.500270,3336.690529,3048.362836,3114.733025,6797.691847,8028.085133,3317.116974,8483.304262,3229.236038,633.671486,3814.195109
New York,3359.383041,2643.050506,8526.194114,4569.460001,327.582714,0.000000,1757.961225,2945.076096,3570.004517,3348.328915,3397.166346,6839.011011,8201.327390,3433.343519,8603.779701,3554.283691,306.108494,4003.222959
Miami,2065.272960,1097.212425,7094.952486,2890.594533,1491.176569,1757.961225,0.000000,2013.994655,1867.122418,1639.742840,1644.533412,5794.758090,6631.402300,2200.658515,7204.245436,2430.007251,2025.336834,2429.837957
Monterrey,705.925938,1213.520720,8028.590573,3711.248568,2623.500270,2945.076096,2013.994655,0.000000,2876.282439,1766.700637,2111.060860,7310.080266,7315.684383,3898.681265,8184.245928,639.142332,3245.755067,3640.297547
Panama City,2408.857647,1715.030837,5333.037977,1025.530802,3336.690529,3570.004517,1867.122418,2876.282439,0.000000,1179.637483,812.486225,4440.058794,4791.336430,1392.559785,5462.750288,2865.971802,3802.437689,766.254430
San Salvador,1233.956557,809.861482,6280.332978,1944.679440,3048.362836,3348.328915,1639.742840,1766.700637,1179.637483,0.000000,367.803223,5601.769099,5613.686901,2451.382655,6430.418383,1687.991760,3634.269046,1937.742276


# Función Objetivo

In [6]:
def y(ciudad1,ciudad2):
    return distancias[ciudad1,ciudad2]

def evaluar(poblacion):
    evaluacion = np.zeros((poblacion.shape[0]))
    for idx,fila in enumerate(poblacion):
        individuo = fila[0]
        n = len(individuo) - 1
        aptitud = 0
        
        for i in range (n,-1,-1):
            c1 = int(individuo[i]) #Id de la ciudad
            c2 = int(individuo[i-1])
            distancia = y(c1,c2)
            aptitud += distancia
        evaluacion[idx] = np.around(aptitud,4)
        
    return evaluacion

# Generación de población

Para poder realizar una buena comparación con las distintas combinaciones de operadores genéticos se trabajará con 5 poblaciones iniciales, las cuales son las mismas parar cada experimento


Si ya existe el archivo de poblaciones no hay necesidad de crear nuevamente las poblaciones iniciales

In [7]:
POB_FILE = "files\poblaciones_3.pkl"

In [8]:
def get_poblaciones_iniciales(n=5, tam_pob=60):
    """
    Genera y carga las poblaciones iniciales para un algoritmo genético.

    Si el archivo de poblaciones no existe, la función lo crea generando las poblaciones iniciales y guardándolas en el archivo.
    Si el archivo ya existe, la función lo carga y devuelve las poblaciones.

    Parámetros:
        n (int): Número de poblaciones iniciales a generar (por defecto, 5).
        tam_pob (int): Tamaño de cada población (por defecto, 60).

    Valor de retorno:
        list: Lista de poblaciones iniciales, donde cada población es una matriz de numpy con 4 columnas: ruta, aptitud, individuo y aptitud.
    """
    poblaciones = []
    POB_PATH = os.path.join("..", POB_FILE)
    if not os.path.exists(POB_PATH):
        print("El archivo de poblaciones no existse, creando archivo")
        
        for i in range(n):
            recorrido = generar_poblacion_perm(ciudades, tam_pob)
            
            poblacion = np.zeros((recorrido.shape[0], 4), dtype="object")
            poblacion[:, 0] = [c.astype(int) for c in recorrido]  # Ruta
            poblacion[:, 1] = np.ones((recorrido.shape[0]))
            poblacion[:,2] = [" ".join([chr(c+64) for c in perm.astype(int)]) for perm in recorrido]
            # poblacion[:, 2] = [individuo_toString(c.astype(int)) for c in recorrido]
            #poblacion[:, 3] = np.zeros((recorrido.shape[0]))  # Aptitud
            poblacion[:, 3] = evaluar(poblacion)
        
            poblaciones.append(poblacion)
        
        try:
            with open(POB_PATH, "wb") as f:
                pickle.dump(poblaciones,f)
            print("Archivo de poblaciones creado correctamente")
        except Exception as e:
            print(f"Error al guardar archivo: {e}")
            
    else:
        try:
            with open(POB_PATH, "rb") as f:
                poblaciones = pickle.load(f)
            print("Archivo de poblaciones cargado correctamente")
        except Exception as e:
            print(f"Error al cargar archivo: {e}")
    
    return poblaciones

In [9]:
pobs = get_poblaciones_iniciales(n=5,tam_pob=100)
print(len(pobs))
print(pobs[0][:2])

Archivo de poblaciones cargado correctamente
5
[[array([ 3,  8,  9, 17, 11,  2,  6, 13, 15, 14,  7, 12,  4,  0,  1, 10, 16,
          5])
  1.0 'C H I Q K B F M O N G L D @ A J P E' 68597.0728]
 [array([14,  9,  2,  7,  6,  3,  8, 15,  1,  5, 10, 16, 17, 13,  0, 11,  4,
         12])
  1.0 'N I B G F C H O A E J P Q M @ K D L' 72336.4905]]


In [10]:
def historial_cruces(descendencia):
    hijos = [ind[0] for ind in descendencia]
    padres_codificados = [ind[1] for ind in descendencia]
    padres_codificados = [
        [p.replace(" ", "") for p in padres] for padres in padres_codificados
    ]
    padres_decodificados = [
        [[ord(c) - 64 for c in p] for p in padres] for padres in padres_codificados
    ]
    
    historial = []
    
    for j in range(0,len(hijos),2):
        fila = np.array([padres_decodificados[j][0],padres_decodificados[j][1],hijos[j],hijos[j+1]])
        historial.append(fila)

    return np.array(historial)

In [11]:
def get_report(n_poblacion,poblacion,time=None):
    info = np.zeros((8))
    info[0] = n_poblacion
    info[1] = np.around(np.mean(poblacion[:,3]),decimals=2)
    info[2] = np.around(np.var(poblacion[:,3]),decimals=2)
    info[3] = np.around(np.std(poblacion[:,3]),decimals=2)
    info[4] = np.around(np.min(poblacion[:,3]),decimals=2)
    info[5] = np.around(np.max(poblacion[:,3]),decimals=2)
    info[6] = np.around(diversity_rate(poblacion),decimals=2)
    if time:
        info[7] = time
    
    return info

In [12]:
def intial_stats(pobs, msg):
    reportes_iniciales = np.array([get_report(i + 1, p) for i, p in enumerate(pobs)])
    # Comportamiento de las poblaciones iniciales
    fig = make_subplots(
        rows=6,
        cols=1,
        subplot_titles=[
            "Costo promedio",
            "Varianza del costo",
            "Desviación estándar",
            "Individuos con menor costo población inicial",
            "Individuos mayor costo población inicial",
            "Diversidad de genes",
        ],
    )

    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:, 0],
            y=reportes_iniciales[:, 1],
            mode="markers+lines",
            name="Costo promedio inicial",
            marker=dict(symbol="star-diamond", size=8),
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:, 0],
            y=reportes_iniciales[:, 2],
            mode="markers+lines",
            marker=dict(symbol="diamond-x", size=8),
            name="Varianza costo",
        ),
        row=2,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:, 0],
            y=reportes_iniciales[:, 3],
            mode="markers+lines",
            marker=dict(symbol="octagon", size=8),
            name="Desviación costo",
        ),
        row=3,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:, 0],
            y=reportes_iniciales[:, 4],
            mode="markers+lines",
            marker=dict(symbol="bowtie", size=8),
            name="Ruta con Menor Distancia",
        ),
        row=4,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:, 0],
            y=reportes_iniciales[:, 5],
            mode="markers+lines",
            marker=dict(symbol="bowtie", size=8),
            name="Ruta con Mayor Distancia",
        ),
        row=5,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:, 0],
            y=reportes_iniciales[:, 6],
            mode="markers+lines",
            marker=dict(symbol="hexagram", size=8),
            name="Costo mayor",
        ),
        row=6,
        col=1,
    )

    fig.update_layout(height=1000, width=900)  # Ajusta el tamaño de la figura
    # Agrega títulos a los ejes x y y de cada gráfica
    fig.update_layout(
        title=f"{msg}",
        xaxis1=dict(title="número población"),  # Eje x de la primera gráfica
        yaxis1=dict(title="Costo"),  # Eje y de la primera gráfica
        xaxis2=dict(title="número población"),  # Eje x de la segunda gráfica
        yaxis2=dict(title="Costo"),  # Eje y de la segunda gráfica
        xaxis3=dict(title="número población"),  # Eje x de la tercera gráfica
        yaxis3=dict(title="Costo"),  # Eje y de la tercera gráfica
        xaxis4=dict(title="número población"),  # Eje x de la tercera gráfica
        yaxis4=dict(title="Costo"),  # Eje y de la tercera gráfica
        xaxis5=dict(title="número población"),  # Eje x de la tercera gráfica
        yaxis6=dict(title="Genes"),  # Eje y de la tercera gráfica
    )
    fig.show()

In [13]:
def show_report(resutaldos, n_gen, total_pobs=5, msg=""):
    gen_reports = np.zeros((total_pobs, n_gen, 8))
    for i, rg in enumerate(resutaldos):
        rg_tiempos = rg[:, 1]
        rg_res = rg[:, 0]

        rg_res_pobs = np.array([respob[0] for respob in rg_res])
        rg_report = np.array(
            [
                get_report(k + 1, rpob, rg_tiempos[k])
                for k, rpob in enumerate(rg_res_pobs)
            ]
        )
        gen_reports[i] = rg_report

    fig = make_subplots(
        rows=3,
        cols=2,
        subplot_titles=[
            "Costo promedio",
            "Varianza",
            "Desviación estándar",
            "Ruta con la Menor Distancia",
            "Ruta con la Mayor Distancia",
            "Diversidad genética",
        ],
    )

    colors = [
        "#f5bde6",  # Pink
        "#c6a0f6",  # Mauve
        "#ed8796",  # Red
        "#a6da95",  # Green
        "#7dc4e4",  # Sapphire
        "#8bd5ca",  # Teal
        "#f5a97f",  # Peach
        "#91d7e3",  # Sky
        "#8aadf4",  # Blue
        "#5b6078"   # Surface
    ]
    
    marker_symbols = [
        "pentagon",
        "diamond",
        "circle",
        "hexagram",
        "star",
        "square",
        "x",
        "star-square",
        "bowtie",
        "circle-x"
    ]
    for i, pop in enumerate(gen_reports):
        fig.add_trace(
            go.Scatter(
                x=pop[:, 0],
                y=pop[:, 1],
                name=f"Población {i+1}",
                mode="markers+lines",
                marker=dict(symbol=marker_symbols[i], size=8, color=colors[i]),
                legendgroup=f"Población {i+1}",
            ),
            row=1,
            col=1,
        )

        fig.add_trace(
            go.Scatter(
                x=pop[:, 0],
                y=pop[:, 2],
                name=f"Población {i+1}",
                mode="markers+lines",
                marker=dict(symbol=marker_symbols[i], size=8, color=colors[i]),
                legendgroup=f"Población {i+1}",
            ),
            row=1,
            col=2,
        )

        fig.add_trace(
            go.Scatter(
                x=pop[:, 0],
                y=pop[:, 3],
                name=f"Población {i+1}",
                mode="markers+lines",
                marker=dict(symbol=marker_symbols[i], size=8, color=colors[i]),
                legendgroup=f"Población {i+1}",
            ),
            row=2,
            col=1,
        )

        fig.add_trace(
            go.Scatter(
                x=pop[:, 0],
                y=pop[:, 4],
                name=f"Población {i+1}",
                mode="markers+lines",
                marker=dict(symbol=marker_symbols[i], size=8, color=colors[i]),
                legendgroup=f"Población {i+1}",
            ),
            row=2,
            col=2,
        )

        fig.add_trace(
            go.Scatter(
                x=pop[:, 0],
                y=pop[:, 5],
                name=f"Población {i+1}",
                mode="markers+lines",
                marker=dict(symbol=marker_symbols[i], size=8, color=colors[i]),
                legendgroup=f"Población {i+1}",
            ),
            row=3,
            col=1,
        )

        fig.add_trace(
            go.Scatter(
                x=pop[:, 0],
                y=pop[:, 6],
                name=f"Población {i+1}",
                mode="markers+lines",
                marker=dict(symbol=marker_symbols[i], size=8, color=colors[i]),
                legendgroup=f"Población {i+1}",
            ),
            row=3,
            col=2,
        )

    for i in range(6):
        fig.update_xaxes(title_text="Generación", row=(i // 2) + 1, col=(i % 2) + 1)

    fig.update_layout(height=1200, width=1500, title=f"{msg}")
    fig.show()

In [14]:
def save_results(res_file_name,reslt):
    res_path = f"../files/act_2/Resultados/{res_file_name}.pkl"
    try:
        with open(res_path,"wb") as f:
            pickle.dump(reslt,f)
    except Exception as e:
        print(f"Error al gaurdar el archvio de resultados: {e}")

In [15]:
def create_excel_hist(reslt,file_name):
    # Colores de resaltado
    fill_green = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")
    fill_yellow = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

    #arch = "exp1_no_mutacion_style2.xlsx"
    hist_file = f"../files/act_2/historial/{file_name}.xlsx"

    with pd.ExcelWriter(hist_file, engine='openpyxl') as writer:
        for i, res in enumerate(reslt):
            start_row = 0

            for g, gen_res in enumerate(res[:, 0]):
                # Crear los DataFrames
                historial_generaciones = np.zeros((gen_res[0][:, 0].shape[0], 2), dtype="object")
                historial_generaciones[:, 0] = np.array([" ".join([str(c) for c in r]) for r in gen_res[0][:, 0]])
                historial_generaciones[:, 1] = gen_res[0][:, 3]
                
                hg_df = pd.DataFrame(historial_generaciones, columns=["Ruta", "Distancia"])

                # DataFrame de cruces con columnas adicionales
                historial_cruces = np.zeros((gen_res[1].shape[0], 4), dtype="object")
                historial_cruces[:, 0] = np.array([" ".join([str(c) for c in r]) for r in gen_res[1][:, 0]])
                historial_cruces[:, 1] = np.array([" ".join([str(c) for c in r]) for r in gen_res[1][:, 1]])
                historial_cruces[:, 2] = np.array([" ".join([str(c) for c in r]) for r in gen_res[1][:, 2]])
                historial_cruces[:, 3] = np.array([" ".join([str(c) for c in r]) for r in gen_res[1][:, 3]])
                
                hc_df = pd.DataFrame(historial_cruces, columns=["Padre 1", "Padre 2", "Hijo 1", "Hijo 2"])
                hc_df["Apto h1"] = hc_df["Hijo 1"].isin(hg_df["Ruta"])
                hc_df["Apto h2"] = hc_df["Hijo 2"].isin(hg_df["Ruta"])

                # DataFrame de mutantes con columna adicional
                historial_mutante = gen_res[2]
                hm_df = pd.DataFrame(historial_mutante, columns=["Original", "Mutante", "Incio mutación", "Genes mutados"])
                hc_df["mutado 1"] = hc_df["Hijo 1"].isin(hm_df["Original"])
                hc_df["mutado 2"] = hc_df["Hijo 2"].isin(hm_df["Original"])
                hm_df["Apto mut"] = hm_df["Mutante"].isin(hg_df["Ruta"])

                # Agregar columnas "Cruza" y "Mutado" a hg_df
                hg_df["Cruza"] = hg_df["Ruta"].isin(hc_df["Hijo 1"]) | hg_df["Ruta"].isin(hc_df["Hijo 2"])
                hg_df["Mutado"] = hg_df["Ruta"].isin(hm_df["Mutante"])

                start_col = 0
                hg_df.to_excel(writer, sheet_name=f"Poblacion_{i}", index=False, startrow=start_row, startcol=start_col)
                start_col += hg_df.shape[1] + 1
                
                hc_df.to_excel(writer, sheet_name=f"Poblacion_{i}", index=False, startrow=start_row, startcol=start_col)
                start_col += hc_df.shape[1] + 1
                
                hm_df.to_excel(writer, sheet_name=f"Poblacion_{i}", index=False, startrow=start_row, startcol=start_col)
                
                start_row += hg_df.shape[0] + 2
                
    # Resaltar las celdas
    wb = load_workbook(hist_file)
    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            ruta_value = row[0].value  # Valor de la columna "Ruta"
            cruza_value = row[2].value  # Valor de la columna "Cruza"
            mutado_value = row[3].value  # Valor de la columna "Mutado"
            
            # Resaltar en verde si "Cruza" es True
            if cruza_value:
                row[0].fill = fill_green
            # Resaltar en amarillo si "Mutado" es True
            if mutado_value:
                row[0].fill = fill_yellow

    wb.save(hist_file)

In [16]:
def seleccion_cruza(poblacion, op_seleccion=0, op_cruza=3, mode="min"):
    desc = get_next_gen(poblacion, op_seleccion, op_cruza, mode)
    cruces = historial_cruces(desc)
    hijos = np.array([hijo[0] for hijo in desc])
    # new_age = poblacion.copy()
    new_age = np.zeros((hijos.shape[0], 4), dtype="object")
    for i, hijo in enumerate(hijos):
        # nuevo_indiviudo = np.zeros((1, 4), dtype="object")
        # nuevo_indiviudo[0, 0] = hijo
        # nuevo_indiviudo[0, 1] = 1.0
        # nuevo_indiviudo[0, 2] = " ".join([chr(c + 64) for c in hijo])
        # nuevo_indiviudo[0, 3] = evaluar(nuevo_indiviudo)[0]
        # new_age = np.vstack([new_age, nuevo_indiviudo])
        new_age[i, 0] = hijo
        new_age[i, 1] = 1.0
        new_age[i, 2] = " ".join([chr(c + 64) for c in hijo])
    # new_age[:, 3] = evaluar(new_age)
    return new_age, cruces


def msg_info_exp(msg):
    print("===================================================================")
    print(f"{msg}")


def msg_info_inicial(num_pob, poblacion):
    print("===================================================================")
    print(f"Población incial #{num_pob+1}")
    print(f"Costo promedio inicial {np.mean(poblacion[:,3])}")


def msg_info_generacion(num_gen, poblacion, tiempo):
    print("...................................................................")
    print(f"Generación: {num_gen}")
    print(f"Costo promedio: {np.mean(poblacion[:,3])}")
    print(f"Varianza del costo en la genearción: {np.var(poblacion[:,3])}")
    print(f"Desviación estándar del costo en la genearción: {np.std(poblacion[:,3])}")
    print(f"Diversidad promedio: {diversity_rate(poblacion)} genes")
    print(f"Menor costo de toda la generación : {np.min(poblacion[:,3])}")
    print(f"Mayor costo de toda la generación : {np.max(poblacion[:,3])}")
    print(f"Tiempo de ejecución de la genaración {num_gen}: {tiempo}")

    print("...................................................................")


def msg_tiempo_pob(num_pob, tiempo):
    print("--------------------------------------------------------------------")
    print(f"Tiempo de ejecución para la población #{num_pob+1}: {tiempo}")
    print("===================================================================")


def experimentacion(
    poblaciones_iniciales,
    max_gen=10,
    op_seleccion=0,
    op_cruza=3,
    mode="min",
    mutar=False,
    n_genes_mutatation = 6,
    op_mutation = 0
):
    start_exp_time = time.time()
    msg_info_exp("Inicio del algoritimo genético")
    gen_results = []

    tam_pob = poblaciones_iniciales[0].shape[0]
    for i in range(len(poblaciones_iniciales)):
        poblacion = poblaciones_iniciales[i]
        # poblacion[:, 3] = evaluar(poblacion)
        poblacion = ordenar_poblacion(poblacion, reverse=True)

        msg_info_inicial(i, poblacion)
        pob_time = 0

        historial_generacional = np.zeros((max_gen, 2), dtype="object")

        for gen in range(max_gen):
            start_gen_time = time.time()
            # if GEN >= MAX_GEN or paro_epsilon(poblacion, 14000, 0.9, opt=1):
            #     print("Convergencia alcanzada")
            #     break

            new_age, cruces = seleccion_cruza(poblacion, op_seleccion, op_cruza, "min")
            mutantes = np.zeros((1, 4))

            if mutar:
                print("mutar")
                pob_mutante, mutantes = mutation(
                    new_age,
                    opt=1,
                    operator=op_mutation,
                    funcion_aptitud=evaluar,
                    cromosomas_mutation=n_genes_mutatation,
                )
                new_age = pob_mutante.copy()

            new_age = np.vstack([poblacion, new_age])
            new_age[:,3] = evaluar(new_age)
            
            poblacion = ordenar_poblacion(new_age, reverse=True)[: tam_pob]
            
            end_gen_time = time.time()
            elapsed_gen_time = end_gen_time - start_gen_time
            pob_time += elapsed_gen_time

            msg_info_generacion(gen, poblacion, elapsed_gen_time)
            historial_generacional[gen, 0] = [poblacion, cruces,mutantes]
            historial_generacional[gen, 1] = elapsed_gen_time

        msg_tiempo_pob(i, pob_time)
        gen_results.append(historial_generacional)
        
    end_exp_time = time.time()
    elapsed_exp_time = end_exp_time - start_exp_time
    msg_info_exp(f"Tiempo de experimentación: {elapsed_exp_time}")

    return gen_results, elapsed_exp_time

# Experimento 1 

> Condiciones generales
>- Cromosomas con 18 genes
>- Método hibrido de paro, 40 generaciones y épsilon
>-  Mutación desde la primera generación
>- 10% de los individuos producto de la cruza se mutan
>- Se mutan los individuos con menor aptitud
>- Se mutan 6 genes
>- Método elitistas de selección, solo los más aptos prevalecen
>

## 5 POBLACIONES CON 100 INDIVIDUOS

In [17]:
POB_FILE = "files\\act_2\\poblaciones\\pob_5_100.pkl"
pobs = get_poblaciones_iniciales(n=5,tam_pob=100)
intial_stats(pobs, f"Estadísticas de las {5} poblaciones con {100} individuos")

Archivo de poblaciones cargado correctamente


### 1 Combinación
> #### Primer combinación de operadores
>- Selección Monogámica
>- Cruza PMX
>- Mutación heurística
>

In [18]:
#5 poblaciones de 100 individuos, con mutacion y primer combinacion de operadores
exp_name1 = "pob_5_100_1comb_mut"
exp_name2 = "pob_5_100_1comb_no_mut"
max_gen_ = 40

In [19]:
exp_res_file = f"../files/act_2/Resultados/{exp_name1}.pkl"
if not os.path.exists(exp_res_file):
    resultados1, tiempo_ejecucion1 = experimentacion(
        poblaciones_iniciales=pobs,
        max_gen=max_gen_,
        op_seleccion=1,
        op_cruza=3,
        mode="min",
        mutar=True,
    )
else:
    try:
        with open(exp_res_file,"rb") as f:
            resultados1 = pickle.load(f)
            tiempo_ejecucion1 = 0
    except Exception as e:
        print(f"Error al cargar el archivo {exp_res_file}: {e}")
    

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 70084.437166
mutar
...................................................................
Generación: 0
Costo promedio: 63050.46867199997
Varianza del costo en la genearción: 35626982.21069016
Desviación estándar del costo en la genearción: 5968.834242185836
Diversidad promedio: 16.919393939393938 genes
Menor costo de toda la generación : 44662.7298
Mayor costo de toda la generación : 69649.2954
Tiempo de ejecución de la genaración 0: 2.8249318599700928
...................................................................
mutar
...................................................................
Generación: 1
Costo promedio: 58881.267063999956
Varianza del costo en la genearción: 26286028.29068902
Desviación estándar del costo en la genearción: 5126.9901785247275
Diversidad promedio: 16.795555555555556 genes
Menor costo de toda la generación : 44662.7298
Mayor costo de toda la generación : 65917.6139
Tiempo de ejecució

In [20]:
create_excel_hist(resultados1,exp_name1)
save_results(exp_name1,resultados1)

In [21]:

exp_res_file = f"../files/act_2/Resultados/{exp_name2}.pkl"
if not os.path.exists(exp_res_file):
    resultados2, tiempo_ejecucion2 = experimentacion(
        poblaciones_iniciales=pobs,
        max_gen=max_gen_,
        op_seleccion=1,
        op_cruza=3,
        mode="min",
        mutar=False,
    )
else:
    try:
        with open(exp_res_file,"rb") as f:
            resultados2 = pickle.load(f)
            tiempo_ejecucion2 = 0
    except Exception as e:
        print(f"Error al cargar el archivo {exp_res_file}: {e}")

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 70084.437166
...................................................................
Generación: 0
Costo promedio: 64280.50769800001
Varianza del costo en la genearción: 33963714.941897415
Desviación estándar del costo en la genearción: 5827.839646206596
Diversidad promedio: 16.897171717171716 genes
Menor costo de toda la generación : 44662.7298
Mayor costo de toda la generación : 70619.5865
Tiempo de ejecución de la genaración 0: 0.039168596267700195
...................................................................
...................................................................
Generación: 1
Costo promedio: 60063.19261599998
Varianza del costo en la genearción: 28395059.838718913
Desviación estándar del costo en la genearción: 5328.701515258564
Diversidad promedio: 16.815959595959598 genes
Menor costo de toda la generación : 44662.7298
Mayor costo de toda la generación : 66003.1838
Tiempo de ejecución de la ge

In [22]:
create_excel_hist(resultados2,exp_name2)
save_results(exp_name2,resultados2)

In [23]:
show_report(resultados1, max_gen_, msg=f"Resultados {exp_name1} mutación")

In [24]:
show_report(resultados2, max_gen_, msg=f"Resultados {exp_name2} sin mutación")

### 2 Combinación
> #### Primer combinación de operadores
>- Selección Monogámica
>- Cruza CX
>- Mutación Scramble
>

In [25]:
#5 poblaciones de 100 individuos, con mutacion y primer combinacion de operadores
exp_name3 = "pob_5_100_2comb_mut"
exp_name4 = "pob_5_100_2comb_no_mut"
max_gen_ = 40

In [26]:
exp_res_file = f"../files/act_2/Resultados/{exp_name3}.pkl"
if not os.path.exists(exp_res_file):
    resultados3, tiempo_ejecucion3 = experimentacion(
        poblaciones_iniciales=pobs,
        max_gen=max_gen_,
        op_seleccion=1,
        op_cruza=4,
        op_mutation=1,
        mode="min",
        mutar=True,
    )
else:
    try:
        with open(exp_res_file, "rb") as f:
            resultados3 = pickle.load(f)
            tiempo_ejecucion3 = 0
    except Exception as e:
        print(f"Error al cargar el archivo {exp_res_file}: {e}")

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 70084.437166
mutar
...................................................................
Generación: 0
Costo promedio: 63695.52762500001
Varianza del costo en la genearción: 31316902.33747182
Desviación estándar del costo en la genearción: 5596.150671441203
Diversidad promedio: 16.86262626262626 genes
Menor costo de toda la generación : 44662.7298
Mayor costo de toda la generación : 70053.2735
Tiempo de ejecución de la genaración 0: 0.03199911117553711
...................................................................
mutar
...................................................................
Generación: 1
Costo promedio: 59991.05544899996
Varianza del costo en la genearción: 21323844.626824845
Desviación estándar del costo en la genearción: 4617.774856662551
Diversidad promedio: 16.74282828282828 genes
Menor costo de toda la generación : 44662.7298
Mayor costo de toda la generación : 65917.6139
Tiempo de ejecución 

In [27]:
create_excel_hist(resultados3,exp_name3)
save_results(exp_name3,resultados3)

In [28]:
exp_res_file = f"../files/act_2/Resultados/{exp_name4}.pkl"
if not os.path.exists(exp_res_file):
    resultados4, tiempo_ejecucion4 = experimentacion(
        poblaciones_iniciales=pobs,
        max_gen=max_gen_,
        op_seleccion=1,
        op_cruza=4,
        op_mutation=1,
        mode="min",
        mutar=False,
    )
else:
    try:
        with open(exp_res_file, "rb") as f:
            resultados4 = pickle.load(f)
            tiempo_ejecucion4 = 0
    except Exception as e:
        print(f"Error al cargar el archivo {exp_res_file}: {e}")

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 70084.437166
...................................................................
Generación: 0
Costo promedio: 63427.977611999995
Varianza del costo en la genearción: 30588052.618246917
Desviación estándar del costo en la genearción: 5530.64667269994
Diversidad promedio: 16.875555555555554 genes
Menor costo de toda la generación : 44662.7298
Mayor costo de toda la generación : 69990.679
Tiempo de ejecución de la genaración 0: 0.03299903869628906
...................................................................
...................................................................
Generación: 1
Costo promedio: 59239.50723599999
Varianza del costo en la genearción: 23463014.727184244
Desviación estándar del costo en la genearción: 4843.863615667172
Diversidad promedio: 16.747474747474747 genes
Menor costo de toda la generación : 44662.7298
Mayor costo de toda la generación : 65840.0694
Tiempo de ejecución de la gena

In [29]:
create_excel_hist(resultados4,exp_name4)
save_results(exp_name4,resultados4)

In [30]:
show_report(resultados3, max_gen_, msg=f"Resultados {exp_name3} mutación")

In [31]:
show_report(resultados4, max_gen_, msg=f"Resultados {exp_name4} mutación")

### 3 Combinación
> #### Tercer combinación de operadores
>- Selección Ruleta
>- Cruza CX
>- Mutación Heuristica

In [32]:
#5 poblaciones de 100 individuos, con mutacion y primer combinacion de operadores
exp_name5 = "pob_5_100_3comb_mut"
exp_name6 = "pob_5_100_3comb_no_mut"
max_gen_ = 40

In [33]:
exp_res_file = f"../files/act_2/Resultados/{exp_name5}.pkl"
if not os.path.exists(exp_res_file):
    resultados5, tiempo_ejecucion5 = experimentacion(
        poblaciones_iniciales=pobs,
        max_gen=max_gen_,
        op_seleccion=0,
        op_cruza=4,
        mode="min",
        mutar=True,
    )
else:
    try:
        with open(exp_res_file, "rb") as f:
            resultados5 = pickle.load(f)
            tiempo_ejecucion5 = 0
    except Exception as e:
        print(f"Error al cargar el archivo {exp_res_file}: {e}")

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 70084.437166
mutar
...................................................................
Generación: 0
Costo promedio: 63007.410736999984
Varianza del costo en la genearción: 33772910.64325731
Desviación estándar del costo en la genearción: 5811.446519005171
Diversidad promedio: 16.7789898989899 genes
Menor costo de toda la generación : 44662.7298
Mayor costo de toda la generación : 69863.2165
Tiempo de ejecución de la genaración 0: 2.6298506259918213
...................................................................
mutar
...................................................................
Generación: 1
Costo promedio: 58868.495085
Varianza del costo en la genearción: 21214729.374879554
Desviación estándar del costo en la genearción: 4605.945003457982
Diversidad promedio: 16.725454545454546 genes
Menor costo de toda la generación : 44662.7298
Mayor costo de toda la generación : 64977.1069
Tiempo de ejecución de la

In [34]:
create_excel_hist(resultados5,exp_name5)
save_results(exp_name5,resultados5)

In [35]:
exp_res_file = f"../files/act_2/Resultados/{exp_name6}.pkl"
if not os.path.exists(exp_res_file):
    resultados6, tiempo_ejecucion6 = experimentacion(
        poblaciones_iniciales=pobs,
        max_gen=max_gen_,
        op_seleccion=0,
        op_cruza=4,
        mode="min",
        mutar=False,
    )
else:
    try:
        with open(exp_res_file, "rb") as f:
            resultados6 = pickle.load(f)
            tiempo_ejecucion6 = 0
    except Exception as e:
        print(f"Error al cargar el archivo {exp_res_file}: {e}")
    

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 70084.437166
...................................................................
Generación: 0
Costo promedio: 63455.73408199994
Varianza del costo en la genearción: 36712473.596958354
Desviación estándar del costo en la genearción: 6059.081910401802
Diversidad promedio: 16.766868686868687 genes
Menor costo de toda la generación : 44662.7298
Mayor costo de toda la generación : 69885.2558
Tiempo de ejecución de la genaración 0: 0.09754228591918945
...................................................................
...................................................................
Generación: 1
Costo promedio: 59348.765725999976
Varianza del costo en la genearción: 24893870.58632808
Desviación estándar del costo en la genearción: 4989.375771209068
Diversidad promedio: 16.584040404040405 genes
Menor costo de toda la generación : 44662.7298
Mayor costo de toda la generación : 65991.3596
Tiempo de ejecución de la gen

In [36]:
create_excel_hist(resultados6,exp_name6)
save_results(exp_name6,resultados6)

In [37]:
show_report(resultados5, max_gen_, msg=f"Resultados {exp_name5} mutación")

In [38]:
show_report(resultados6, max_gen_, msg=f"Resultados {exp_name6} mutación")

### 4 Combinación

> #### Cuarta combinación de operadores
>- Selección Ruleta
>- Cruza PMX
>- Mutación Scramble


In [39]:
#5 poblaciones de 100 individuos, con mutacion y primer combinacion de operadores
exp_name7 = "pob_5_100_4comb_mut"
exp_name8 = "pob_5_100_4comb_no_mut"
max_gen_ = 40

In [40]:
exp_res_file = f"../files/act_2/Resultados/{exp_name7}.pkl"
if not os.path.exists(exp_res_file):
    resultados7, tiempo_ejecucion7 = experimentacion(
        poblaciones_iniciales=pobs,
        max_gen=max_gen_,
        op_seleccion=0,
        op_cruza=3,
        op_mutation=1, 
        mode="min",
        mutar=True,
    )
else:
    try:
        with open(exp_res_file,"rb") as f:
            resultados7 = pickle.load(f)
            tiempo_ejecucion7 = 0
    except Exception as e:
        print(f"Error al cargar el archivo {exp_res_file}: {e}")

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 70084.437166
mutar
...................................................................
Generación: 0
Costo promedio: 64101.63709599998
Varianza del costo en la genearción: 30560063.047536086
Desviación estándar del costo en la genearción: 5528.115686880665
Diversidad promedio: 16.863434343434342 genes
Menor costo de toda la generación : 44662.7298
Mayor costo de toda la generación : 70299.8675
Tiempo de ejecución de la genaración 0: 0.09751629829406738
...................................................................
mutar
...................................................................
Generación: 1
Costo promedio: 59722.86131599999
Varianza del costo en la genearción: 29751054.873154603
Desviación estándar del costo en la genearción: 5454.452756524214
Diversidad promedio: 16.713535353535352 genes
Menor costo de toda la generación : 41248.5096
Mayor costo de toda la generación : 66003.1838
Tiempo de ejecuci

In [41]:
create_excel_hist(resultados7,exp_name7)
save_results(exp_name7,resultados7)

In [42]:
exp_res_file = f"../files/act_2/Resultados/{exp_name8}.pkl"
if not os.path.exists(exp_res_file):
    resultados8, tiempo_ejecucion8 = experimentacion(
        poblaciones_iniciales=pobs,
        max_gen=max_gen_,
        op_seleccion=0,
        op_cruza=3,
        op_mutation=1, 
        mode="min",
        mutar=True,
    )
else:
    try:
        with open(exp_res_file,"rb") as f:
            resultados8 = pickle.load(f)
            tiempo_ejecucion8 = 0
    except Exception as e:
        print(f"Error al cargar el archivo {exp_res_file}: {e}")

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 70084.437166
mutar
...................................................................
Generación: 0
Costo promedio: 63241.53535899999
Varianza del costo en la genearción: 33343185.26764032
Desviación estándar del costo en la genearción: 5774.355831401484
Diversidad promedio: 16.854343434343434 genes
Menor costo de toda la generación : 44662.7298
Mayor costo de toda la generación : 69661.8054
Tiempo de ejecución de la genaración 0: 0.10051822662353516
...................................................................
mutar
...................................................................
Generación: 1
Costo promedio: 59315.630961999996
Varianza del costo en la genearción: 30292638.82005297
Desviación estándar del costo en la genearción: 5503.8748913881545
Diversidad promedio: 16.84141414141414 genes
Menor costo de toda la generación : 44126.9812
Mayor costo de toda la generación : 65655.2389
Tiempo de ejecució

In [43]:
create_excel_hist(resultados8,exp_name8)
save_results(exp_name8,resultados8)

In [44]:
show_report(resultados7, max_gen_, msg=f"Resultados {exp_name7} mutación")

In [45]:
show_report(resultados8, max_gen_, msg=f"Resultados {exp_name8} mutación")

## 7 POBLACIONES CON 100 INDIVIDUOS

In [46]:
POB_FILE = "files\\act_2\\poblaciones\\pob_7_100.pkl"
pobs = get_poblaciones_iniciales(n=7,tam_pob=100)
intial_stats(pobs,f"Estadísticas de las {7} poblaciones con {100} individuos")

Archivo de poblaciones cargado correctamente


### 1 Combinación
> #### Primer combinación de operadores
>- Selección Monogámica
>- Cruza PMX
>- Mutación heurística
>

In [47]:
#7 poblaciones de 100 individuos, con mutacion y primer combinacion de operadores
exp_name9 = "pob_7_100_1comb_mut"
exp_name10 = "pob_7_100_1comb_no_mut"
max_gen_ = 40

In [48]:
exp_res_file = f"../files/act_2/Resultados/{exp_name9}.pkl"
if not os.path.exists(exp_res_file):
    resultados9, tiempo_ejecucion9 = experimentacion(
        poblaciones_iniciales=pobs,
        max_gen=max_gen_,
        op_seleccion=1,
        op_cruza=3,
        mode="min",
        mutar=True,
    )
else:
    try:
        with open(exp_res_file,"rb") as f:
            resultados9 = pickle.load(f)
            tiempo_ejecucion9 = 0
    except Exception as e:
        print(f"Error al cargar el archivo {exp_res_file}: {e}")
    

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 69123.28244
mutar
...................................................................
Generación: 0
Costo promedio: 61576.64380600001
Varianza del costo en la genearción: 36691141.80314173
Desviación estándar del costo en la genearción: 6057.321338937017
Diversidad promedio: 16.96121212121212 genes
Menor costo de toda la generación : 45536.3036
Mayor costo de toda la generación : 69601.5119
Tiempo de ejecución de la genaración 0: 2.720064163208008
...................................................................
mutar
...................................................................
Generación: 1
Costo promedio: 56961.430854999984
Varianza del costo en la genearción: 23305581.876510944
Desviación estándar del costo en la genearción: 4827.585512086859
Diversidad promedio: 16.886868686868688 genes
Menor costo de toda la generación : 42006.1211
Mayor costo de toda la generación : 63109.6691
Tiempo de ejecución d

In [49]:
create_excel_hist(resultados9,exp_name9)
save_results(exp_name9,resultados9)

In [50]:
exp_res_file = f"../files/act_2/Resultados/{exp_name10}.pkl"
if not os.path.exists(exp_res_file):
    resultados10, tiempo_ejecucion10 = experimentacion(
        poblaciones_iniciales=pobs,
        max_gen=max_gen_,
        op_seleccion=1,
        op_cruza=3,
        mode="min",
        mutar=False,
    )
else:
    try:
        with open(exp_res_file,"rb") as f:
            resultados10 = pickle.load(f)
            tiempo_ejecucion10 = 0
    except Exception as e:
        print(f"Error al cargar el archivo {exp_res_file}: {e}")

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 69123.28244
...................................................................
Generación: 0
Costo promedio: 62952.504401000006
Varianza del costo en la genearción: 26616461.460399136
Desviación estándar del costo en la genearción: 5159.114406601111
Diversidad promedio: 16.94181818181818 genes
Menor costo de toda la generación : 45536.3036
Mayor costo de toda la generación : 69775.5181
Tiempo de ejecución de la genaración 0: 0.027001380920410156
...................................................................
...................................................................
Generación: 1
Costo promedio: 59533.001380999995
Varianza del costo en la genearción: 14771088.279401641
Desviación estándar del costo en la genearción: 3843.3173534593316
Diversidad promedio: 16.83939393939394 genes
Menor costo de toda la generación : 45536.3036
Mayor costo de toda la generación : 64363.0664
Tiempo de ejecución de la ge

In [51]:
create_excel_hist(resultados10,exp_name10)
save_results(exp_name10,resultados10)

In [52]:
show_report(resultados9, max_gen_, msg=f"Resultados {exp_name9} mutación",total_pobs=7)

In [53]:
show_report(resultados10, max_gen_, msg=f"Resultados {exp_name10} sin mutación",total_pobs=7)

### 2 Combinación
> #### Segunda combinación de operadores
>- Selección Monogámica
>- Cruza CX
>- Mutación Scramble
>

In [54]:
#7 poblaciones de 100 individuos, con mutacion y primer combinacion de operadores
exp_name11 = "pob_7_100_2comb_mut"
exp_name12 = "pob_7_100_2comb_no_mut"
max_gen_ = 40

In [55]:
exp_res_file = f"../files/act_2/Resultados/{exp_name11}.pkl"
if not os.path.exists(exp_res_file):
    resultados11, tiempo_ejecucion11 = experimentacion(
        poblaciones_iniciales=pobs,
        max_gen=max_gen_,
        op_seleccion=1,
        op_cruza=4,
        op_mutation=1,
        mode="min",
        mutar=True,
    )
else:
    try:
        with open(exp_res_file, "rb") as f:
            resultados11 = pickle.load(f)
            tiempo_ejecucion11 = 0
    except Exception as e:
        print(f"Error al cargar el archivo {exp_res_file}: {e}")

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 69123.28244
mutar
...................................................................
Generación: 0
Costo promedio: 62731.212945000014
Varianza del costo en la genearción: 26712544.507195953
Desviación estándar del costo en la genearción: 5168.417988823655
Diversidad promedio: 16.94 genes
Menor costo de toda la generación : 45536.3036
Mayor costo de toda la generación : 69775.5181
Tiempo de ejecución de la genaración 0: 0.027995586395263672
...................................................................
mutar
...................................................................
Generación: 1
Costo promedio: 59174.36150999998
Varianza del costo en la genearción: 19152361.22100129
Desviación estándar del costo en la genearción: 4376.34107685876
Diversidad promedio: 16.77090909090909 genes
Menor costo de toda la generación : 43654.091
Mayor costo de toda la generación : 64411.499
Tiempo de ejecución de la genaraci

In [56]:
create_excel_hist(resultados11,exp_name11)
save_results(exp_name11,resultados11)

In [57]:
exp_res_file = f"../files/act_2/Resultados/{exp_name12}.pkl"
if not os.path.exists(exp_res_file):
    resultados12, tiempo_ejecucion12 = experimentacion(
        poblaciones_iniciales=pobs,
        max_gen=max_gen_,
        op_seleccion=1,
        op_cruza=4,
        op_mutation=1,
        mode="min",
        mutar=False,
    )
else:
    try:
        with open(exp_res_file, "rb") as f:
            resultados12 = pickle.load(f)
            tiempo_ejecucion12 = 0
    except Exception as e:
        print(f"Error al cargar el archivo {exp_res_file}: {e}")

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 69123.28244
...................................................................
Generación: 0
Costo promedio: 62462.774756000006
Varianza del costo en la genearción: 29761447.854059197
Desviación estándar del costo en la genearción: 5455.405379443328
Diversidad promedio: 16.922020202020203 genes
Menor costo de toda la generación : 45536.3036
Mayor costo de toda la generación : 69066.011
Tiempo de ejecución de la genaración 0: 0.022000789642333984
...................................................................
...................................................................
Generación: 1
Costo promedio: 57922.25115
Varianza del costo en la genearción: 18384557.038602125
Desviación estándar del costo en la genearción: 4287.721660579442
Diversidad promedio: 16.76969696969697 genes
Menor costo de toda la generación : 45536.3036
Mayor costo de toda la generación : 64152.1021
Tiempo de ejecución de la genaración

In [58]:
create_excel_hist(resultados12,exp_name12)
save_results(exp_name12,resultados12)

In [59]:
show_report(resultados11, max_gen_, msg=f"Resultados {exp_name11} mutación",total_pobs=7)

In [60]:
show_report(resultados12, max_gen_, msg=f"Resultados {exp_name12} mutación",total_pobs=7)

### 3 Combinación
> #### Tercer combinación de operadores
>- Selección Ruleta
>- Cruza CX
>- Mutación Heuristica

In [61]:
#7 poblaciones de 100 individuos, con mutacion y primer combinacion de operadores
exp_name13 = "pob_7_100_3comb_mut"
exp_name14 = "pob_7_100_3comb_no_mut"
max_gen_ = 40

In [62]:
exp_res_file = f"../files/act_2/Resultados/{exp_name13}.pkl"
if not os.path.exists(exp_res_file):
    resultados13, tiempo_ejecucion13 = experimentacion(
        poblaciones_iniciales=pobs,
        max_gen=max_gen_,
        op_seleccion=0,
        op_cruza=4,
        mode="min",
        mutar=True,
    )
else:
    try:
        with open(exp_res_file, "rb") as f:
            resultados13 = pickle.load(f)
            tiempo_ejecucion13 = 0
    except Exception as e:
        print(f"Error al cargar el archivo {exp_res_file}: {e}")

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 69123.28244
mutar
...................................................................
Generación: 0
Costo promedio: 61342.623921
Varianza del costo en la genearción: 29763724.57122072
Desviación estándar del costo en la genearción: 5455.614041629111
Diversidad promedio: 16.873131313131314 genes
Menor costo de toda la generación : 43709.6219
Mayor costo de toda la generación : 68403.8158
Tiempo de ejecución de la genaración 0: 2.5115606784820557
...................................................................
mutar
...................................................................
Generación: 1
Costo promedio: 57767.52680799997
Varianza del costo en la genearción: 19498925.843464065
Desviación estándar del costo en la genearción: 4415.7588072112885
Diversidad promedio: 16.683030303030304 genes
Menor costo de toda la generación : 43709.6219
Mayor costo de toda la generación : 62609.8794
Tiempo de ejecución de l

In [63]:
create_excel_hist(resultados13,exp_name13)
save_results(exp_name13,resultados13)

In [64]:
exp_res_file = f"../files/act_2/Resultados/{exp_name14}.pkl"
if not os.path.exists(exp_res_file):
    resultados14, tiempo_ejecucion14 = experimentacion(
        poblaciones_iniciales=pobs,
        max_gen=max_gen_,
        op_seleccion=0,
        op_cruza=4,
        mode="min",
        mutar=False,
    )
else:
    try:
        with open(exp_res_file, "rb") as f:
            resultados14 = pickle.load(f)
            tiempo_ejecucion14 = 0
    except Exception as e:
        print(f"Error al cargar el archivo {exp_res_file}: {e}")

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 69123.28244
...................................................................
Generación: 0
Costo promedio: 60883.446831
Varianza del costo en la genearción: 34678690.15863122
Desviación estándar del costo en la genearción: 5888.861533321294
Diversidad promedio: 16.845454545454544 genes
Menor costo de toda la generación : 40766.9823
Mayor costo de toda la generación : 68105.3059
Tiempo de ejecución de la genaración 0: 0.09152054786682129
...................................................................
...................................................................
Generación: 1
Costo promedio: 56870.11953499999
Varianza del costo en la genearción: 26127089.0559985
Desviación estándar del costo en la genearción: 5111.466429117822
Diversidad promedio: 16.56989898989899 genes
Menor costo de toda la generación : 40766.9823
Mayor costo de toda la generación : 63233.0154
Tiempo de ejecución de la genaración 1:

In [65]:
create_excel_hist(resultados14,exp_name14)
save_results(exp_name14,resultados14)

In [66]:
show_report(resultados13, max_gen_, msg=f"Resultados {exp_name13} mutación",total_pobs=7)

In [67]:
show_report(resultados14, max_gen_, msg=f"Resultados {exp_name14} mutación",total_pobs=7)

### 4 Combinación

> #### Cuarta combinación de operadores
>- Selección Ruleta
>- Cruza PMX
>- Mutación Scramble

In [68]:
#7 poblaciones de 100 individuos, con mutacion y primer combinacion de operadores
exp_name15 = "pob_7_100_4comb_mut"
exp_name16 = "pob_7_100_4comb_no_mut"
max_gen_ = 40

In [69]:
exp_res_file = f"../files/act_2/Resultados/{exp_name15}.pkl"
if not os.path.exists(exp_res_file):
    resultados15, tiempo_ejecucion15 = experimentacion(
        poblaciones_iniciales=pobs,
        max_gen=max_gen_,
        op_seleccion=0,
        op_cruza=3,
        op_mutation=1, 
        mode="min",
        mutar=True,
    )
else:
    try:
        with open(exp_res_file,"rb") as f:
            resultados15 = pickle.load(f)
            tiempo_ejecucion15 = 0
    except Exception as e:
        print(f"Error al cargar el archivo {exp_res_file}: {e}")

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 69123.28244
mutar
...................................................................
Generación: 0
Costo promedio: 62298.068422999975
Varianza del costo en la genearción: 32656592.628013387
Desviación estándar del costo en la genearción: 5714.594703740011
Diversidad promedio: 16.904444444444444 genes
Menor costo de toda la generación : 45536.3036
Mayor costo de toda la generación : 69628.2257
Tiempo de ejecución de la genaración 0: 0.09813761711120605
...................................................................
mutar
...................................................................
Generación: 1
Costo promedio: 57861.05044299999
Varianza del costo en la genearción: 19756865.45597024
Desviación estándar del costo en la genearción: 4444.869565686966
Diversidad promedio: 16.771313131313132 genes
Menor costo de toda la generación : 45536.3036
Mayor costo de toda la generación : 63958.5207
Tiempo de ejecució

In [70]:
create_excel_hist(resultados15,exp_name15)
save_results(exp_name15,resultados15)

In [71]:
exp_res_file = f"../files/act_2/Resultados/{exp_name16}.pkl"
if not os.path.exists(exp_res_file):
    resultados16, tiempo_ejecucion16 = experimentacion(
        poblaciones_iniciales=pobs,
        max_gen=max_gen_,
        op_seleccion=0,
        op_cruza=3,
        op_mutation=1, 
        mode="min",
        mutar=True,
    )
else:
    try:
        with open(exp_res_file,"rb") as f:
            resultados16 = pickle.load(f)
            tiempo_ejecucion16 = 0
    except Exception as e:
        print(f"Error al cargar el archivo {exp_res_file}: {e}")

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 69123.28244
mutar
...................................................................
Generación: 0
Costo promedio: 62872.11585
Varianza del costo en la genearción: 29518487.62309606
Desviación estándar del costo en la genearción: 5433.091902691879
Diversidad promedio: 16.91979797979798 genes
Menor costo de toda la generación : 44186.0841
Mayor costo de toda la generación : 69628.2257
Tiempo de ejecución de la genaración 0: 0.09747815132141113
...................................................................
mutar
...................................................................
Generación: 1
Costo promedio: 58857.75517599997
Varianza del costo en la genearción: 24348512.174614605
Desviación estándar del costo en la genearción: 4934.4211590230725
Diversidad promedio: 16.855151515151515 genes
Menor costo de toda la generación : 44186.0841
Mayor costo de toda la generación : 65325.0528
Tiempo de ejecución de la

In [72]:
create_excel_hist(resultados16,exp_name16)
save_results(exp_name16,resultados16)

In [73]:
show_report(resultados15, max_gen_, msg=f"Resultados {exp_name15} mutación",total_pobs=7)

In [74]:
show_report(resultados16, max_gen_, msg=f"Resultados {exp_name16} mutación",total_pobs=7)